### Задание №1.
Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        return pd.get_dummies(X, prefix=self.key)[self.columns]

In [3]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)

In [4]:
df = pd.read_csv('train_case2.csv', ';')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], random_state=23)

__Логистическая регрессия:__

In [6]:
lr = Pipeline([('features', feats),
               ('classifier', LogisticRegression(random_state=23))])

cv_scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.78840 +- 0.00591


__Случайный лес:__

In [7]:
rfc = Pipeline([('features', feats),
                ('classifier', RandomForestClassifier(n_estimators=80,
                                                      max_depth=9,
                                                      random_state=23))])

cv_scores = cross_val_score(rfc, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.80231 +- 0.00460


__Градиентный бустинг:__

In [8]:
gbc = Pipeline([('features', feats),
                ('classifier', GradientBoostingClassifier(learning_rate=0.1,
                                                          n_estimators=110,
                                                          max_depth=4,
                                                          random_state=23))])

cv_scores = cross_val_score(gbc, X_train, y_train, cv=5, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score is {cv_score:.5f} +- {cv_score_std:.5f}')

CV score is 0.80409 +- 0.00490


___

### Задание №2.
Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики).

In [9]:
pivot_metric_table = []
models_pred_proba_values = []

for model in ['lr', 'rfc', 'gbc']:
    eval(f'{model}.fit(X_train, y_train)')
    preds = eval(f'{model}.predict_proba(X_test)[:, 1]')
    models_pred_proba_values.append(preds)
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    precision[(precision == 0) & (recall == 0)] = np.e-10
    fscore = (2 * precision * recall) / (precision + recall)
    
    ix = np.argmax(fscore)
    
    pivot_metric_table.append((fscore[ix], precision[ix], recall[ix], thresholds[ix], roc_auc_score(y_test, preds)))

pivot_metric_table = pd.DataFrame(pivot_metric_table)
pivot_metric_table.columns = ['F score', 'Precision', 'Recall', 'Optimal threshold', 'ROC AUC score']
pivot_metric_table['Classifier'] = ['Logistic Regression', 'Random Forest', 'Gradient Boosting']
pivot_metric_table.set_index('Classifier', inplace=True)
pivot_metric_table

,F score,Precision,Recall,Optimal threshold,ROC AUC score
Classifier,,,,,
Logistic Regression,0.735101,0.657171,0.834001,0.391695,0.783973
Random Forest,0.740866,0.670216,0.828166,0.367101,0.796581
Gradient Boosting,0.741637,0.667399,0.834458,0.346801,0.799145


___

### Задание №3.
Вывести сравнение полученных моделей по метрикам бизнеса по показателям с урока:
   - стоимость лечения 15000р, если сделали тест и начали лечить вовремя;
   - стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы;
   - стоимость теста 1400р.

In [10]:
def get_business_costs(y_test, y_preds, treshold=0.5,
                       test_cost=1400, early_treatment_cost=15000, late_treatment_cost=20000):
    
    cnf_matrix = confusion_matrix(y_test, y_preds > treshold)
    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]
    
    late_costs = (FN + TP) * late_treatment_cost
    test_all_costs = np.sum(cnf_matrix) * test_cost + (FN + TP) * early_treatment_cost
    ml_costs = (FP + TP) * test_cost + FN * late_treatment_cost + TP * early_treatment_cost
    
    return late_costs, test_all_costs, ml_costs

In [11]:
costs_lr = get_business_costs(y_test, models_pred_proba_values[0], pivot_metric_table.iloc[0, 3])
costs_rfc = get_business_costs(y_test, models_pred_proba_values[1], pivot_metric_table.iloc[1, 3])
costs_gbc = get_business_costs(y_test, models_pred_proba_values[2], pivot_metric_table.iloc[2, 3])

___

### Задание №4.
Сделать выводы о том, какая модель справилась с задачей лучше других.

In [12]:
print(f'Прибыль от применения "Логистической регрессии" относительно вариантов:\n'
      f'"без тестов": {costs_lr[0] - costs_lr[2]} р.\n'
      f'"тесты всем": {costs_lr[1] - costs_lr[2]} р.\n')
print(f'Прибыль от применения "Случайного леса" относительно вариантов:\n'
      f'"без тестов": {costs_rfc[0] - costs_rfc[2]} р.\n'
      f'"тесты всем": {costs_rfc[1] - costs_rfc[2]} р.\n')
print(f'Прибыль от применения "Градиентного бустинга" относительно вариантов:\n'
      f'"без тестов": {costs_gbc[0] - costs_gbc[2]} р.\n'
      f'"тесты всем": {costs_gbc[1] - costs_gbc[2]} р.')

Прибыль от применения "Логистической регрессии" относительно вариантов:
"без тестов": 20916200 р.
"тесты всем": 1711200 р.

Прибыль от применения "Случайного леса" относительно вариантов:
"без тестов": 21070000 р.
"тесты всем": 1865000 р.

Прибыль от применения "Градиентного бустинга" относительно вариантов:
"без тестов": 21165800 р.
"тесты всем": 1960800 р.


Как видим из прибыли: лучше всего показала себя модель классификатора методом градиентного бустинга. Отрыв по прибыли от второй по качеству модели, случайного леса: 95800 рублей.

___

### Задание №5.
Найти порог классификации по деньгам для лучшей модели:
   - Стоимость лечения 15000р, если сделали тест и начали лечить вовремя;
   - Стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы;
   - Стоимость теста 1400р.

In [13]:
def get_optimal_business_th(y_test, y_preds, n_thresholds=20,
                            test_cost=1400, early_treatment_cost=15000, late_treatment_cost=20000):
    best_profit = 0
    opt_th = None
    best_costs = None
    
    for th in np.linspace(0, 1, n_thresholds + 1):
        
        cnf_matrix = confusion_matrix(y_test, y_preds > th)
        TN = cnf_matrix[0][0]
        FN = cnf_matrix[1][0]
        TP = cnf_matrix[1][1]
        FP = cnf_matrix[0][1]
    
        late_costs = (FN + TP) * late_treatment_cost
        test_all_costs = np.sum(cnf_matrix) * test_cost + (FN + TP) * early_treatment_cost
        ml_costs = (FP + TP) * test_cost + FN * late_treatment_cost + TP * early_treatment_cost
        
        if (test_all_costs - ml_costs) > best_profit:
            opt_th = th
            best_profit = test_all_costs - ml_costs
            best_costs = late_costs, test_all_costs, ml_costs
    
    return opt_th, best_profit, best_costs

In [14]:
opt_th = get_optimal_business_th(y_test, models_pred_proba_values[2], 100)

In [15]:
print(f'Максимальная прибыль от ML относительно варианта тестирования всех при пороге {opt_th[0]}: {opt_th[1]} рублей.')
print(f'Аналогичная прибыль от ML при пороге {pivot_metric_table.iloc[2, 3]:.2f} (лучшем согласно f_score): {costs_gbc[1] - costs_gbc[2]} рублей.')

Максимальная прибыль от ML относительно варианта тестирования всех при пороге 0.26: 2221400 рублей.
Аналогичная прибыль от ML при пороге 0.35 (лучшем согласно f_score): 1960800 рублей.
